In [1]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
# Load files
data_path = ''
train = pd.read_csv(data_path + 'Train.csv')
test = pd.read_csv(data_path + 'Test.csv')
#sample_submission = pd.read_csv(data_path + 'SampleSubmission.csv')
var_desc = pd.read_csv(data_path + 'VariableDescription.csv')

In [3]:
df = pd.DataFrame(train)
# Check for null values in each column
null_values = df.isnull().sum()

# Display the result
print("Number of null values in each column:")
print(null_values)

Number of null values in each column:
ID                                       0
District                                 0
Block                                    0
CultLand                                 0
CropCultLand                             0
LandPreparationMethod                    0
CropTillageDate                          0
CropTillageDepth                         0
CropEstMethod                            0
RcNursEstDate                           83
SeedingSowingTransplanting               0
SeedlingsPerPit                        289
NursDetFactor                          289
TransDetFactor                         289
TransplantingIrrigationHours           193
TransplantingIrrigationSource          115
TransplantingIrrigationPowerSource     503
TransIrriCost                          882
StandingWater                          238
OrgFertilizers                        1335
Ganaura                               2417
CropOrgFYM                            2674
PCropSolidOrgFer

In [4]:
columns_to_remove = ['Harv_date', 'Threshing_date','RcNursEstDate']
df = df.drop(columns=columns_to_remove, axis=1)

In [5]:
test = test.drop(columns=columns_to_remove, axis=1)

In [6]:
df.head()

,ID,District,Block,CultLand,CropCultLand,LandPreparationMethod,CropTillageDate,CropTillageDepth,CropEstMethod,SeedingSowingTransplanting,...,2appDaysUrea,MineralFertAppMethod.1,Harv_method,Harv_hand_rent,Threshing_method,Residue_length,Residue_perc,Stubble_use,Acre,Yield
0,ID_GTFAC7PEVWQ9,Nalanda,Noorsarai,45,40,TractorPlough FourWheelTracRotavator,2022-07-20,5,Manual_PuddledRandom,2022-07-21,...,NaN,Broadcasting,machine,NaN,machine,30,40,plowed_in_soil,0.312500,600
1,ID_TK40ARLSPOKS,Nalanda,Rajgir,26,26,WetTillagePuddling TractorPlough FourWheelTrac...,2022-07-18,5,Manual_PuddledRandom,2022-07-20,...,NaN,Broadcasting,hand,3.0,machine,24,10,plowed_in_soil,0.312500,600
2,ID_1FJY2CRIMLZZ,Gaya,Gurua,10,10,TractorPlough FourWheelTracRotavator,2022-06-30,6,Manual_PuddledRandom,2022-08-13,...,NaN,RootApplication,hand,480.0,machine,30,10,plowed_in_soil,0.148148,225
3,ID_I3IPXS4DB7NE,Gaya,Gurua,15,15,TractorPlough FourWheelTracRotavator,2022-06-16,6,Manual_PuddledRandom,2022-07-17,...,NaN,Broadcasting,hand,240.0,hand,26,10,plowed_in_soil,0.222222,468
4,ID_4T8YQWXWHB4A,Nalanda,Noorsarai,60,60,TractorPlough WetTillagePuddling,2022-07-19,4,Manual_PuddledRandom,2022-07-20,...,NaN,Broadcasting,machine,NaN,machine,24,40,plowed_in_soil,0.468750,550


In [7]:
# Threshold for null values
threshold = 500

# Original columns
original_columns = df.columns.tolist()

# Drop columns with null values greater than the threshold
df_cleaned = df.dropna(axis=1, thresh=df.shape[0] - threshold+1)

# Updated columns after removal
updated_columns = df_cleaned.columns.tolist()

# Identify removed columns
removed_columns = list(set(original_columns) - set(updated_columns))

# Print the list of removed columns
print("Removed columns:", removed_columns)


Removed columns: ['1appDaysUrea', 'PCropSolidOrgFertAppMethod', '2tdUrea', 'CropOrgFYM', 'TransplantingIrrigationPowerSource', 'Ganaura', '2appDaysUrea', 'BasalUrea', 'OrgFertilizers', '1tdUrea', 'TransIrriCost', 'BasalDAP']


In [8]:
# Remove columns from df_test that are present in removed_columns
df_test_cleaned = test.drop(columns=removed_columns, errors='ignore')

In [9]:
# Remove columns from df_test that are present in removed_columns
df_cleaned = df_cleaned.drop(columns=removed_columns, errors='ignore')

In [10]:
df_test_cleaned.columns

Index(['ID', 'District', 'Block', 'CultLand', 'CropCultLand',
       'LandPreparationMethod', 'CropTillageDate', 'CropTillageDepth',
       'CropEstMethod', 'SeedingSowingTransplanting', 'SeedlingsPerPit',
       'NursDetFactor', 'TransDetFactor', 'TransplantingIrrigationHours',
       'TransplantingIrrigationSource', 'StandingWater', 'NoFertilizerAppln',
       'CropbasalFerts', 'MineralFertAppMethod', 'FirstTopDressFert',
       'MineralFertAppMethod.1', 'Harv_method', 'Harv_hand_rent',
       'Threshing_method', 'Residue_length', 'Residue_perc', 'Stubble_use',
       'Acre'],
      dtype='object')

In [11]:
df_cleaned.columns

Index(['ID', 'District', 'Block', 'CultLand', 'CropCultLand',
       'LandPreparationMethod', 'CropTillageDate', 'CropTillageDepth',
       'CropEstMethod', 'SeedingSowingTransplanting', 'SeedlingsPerPit',
       'NursDetFactor', 'TransDetFactor', 'TransplantingIrrigationHours',
       'TransplantingIrrigationSource', 'StandingWater', 'NoFertilizerAppln',
       'CropbasalFerts', 'MineralFertAppMethod', 'FirstTopDressFert',
       'MineralFertAppMethod.1', 'Harv_method', 'Harv_hand_rent',
       'Threshing_method', 'Residue_length', 'Residue_perc', 'Stubble_use',
       'Acre', 'Yield'],
      dtype='object')

In [12]:
df_test_cleaned.head()

,ID,District,Block,CultLand,CropCultLand,LandPreparationMethod,CropTillageDate,CropTillageDepth,CropEstMethod,SeedingSowingTransplanting,...,MineralFertAppMethod,FirstTopDressFert,MineralFertAppMethod.1,Harv_method,Harv_hand_rent,Threshing_method,Residue_length,Residue_perc,Stubble_use,Acre
0,ID_F9XXEXN2ADR2,Jamui,Khaira,20,13,TractorPlough,2022-07-24,4,Manual_PuddledLine,2022-07-29,...,Broadcasting,Urea,Broadcasting,hand,500.0,hand,22,10,plowed_in_soil,0.272727
1,ID_SO3VW2X4QO93,Jamui,Khaira,25,25,TractorPlough,2022-07-24,5,Manual_PuddledRandom,2022-08-02,...,Broadcasting,Urea,Broadcasting,hand,150.0,machine,28,10,plowed_in_soil,0.227273
2,ID_UKUQ7JM8E894,Nalanda,Rajgir,30,30,WetTillagePuddling TractorPlough FourWheelTrac...,2022-07-07,5,Manual_PuddledRandom,2022-07-08,...,Broadcasting,Urea,Broadcasting,hand,1200.0,machine,28,10,plowed_in_soil,0.281250
3,ID_QUISMWEZR2H4,Vaishali,Mahua,15,10,WetTillagePuddling TractorPlough BullockPlough...,2022-07-04,5,Manual_PuddledRandom,2022-07-10,...,SoilApplied,Urea,SoilApplied,hand,300.0,hand,30,10,plowed_in_soil,0.136364
4,ID_25JGI455VKCZ,Nalanda,Rajgir,30,10,WetTillagePuddling TractorPlough FourWheelTrac...,2022-07-21,3,Manual_PuddledRandom,2022-07-22,...,Broadcasting,Urea,Broadcasting,machine,NaN,machine,26,40,plowed_in_soil,0.312500


In [13]:
# Check for null values in each column
null_values = df_cleaned.isnull().sum()

# Display the result
print("Number of null values in each column:")
print(null_values)

Number of null values in each column:
ID                                 0
District                           0
Block                              0
CultLand                           0
CropCultLand                       0
LandPreparationMethod              0
CropTillageDate                    0
CropTillageDepth                   0
CropEstMethod                      0
SeedingSowingTransplanting         0
SeedlingsPerPit                  289
NursDetFactor                    289
TransDetFactor                   289
TransplantingIrrigationHours     193
TransplantingIrrigationSource    115
StandingWater                    238
NoFertilizerAppln                  0
CropbasalFerts                   188
MineralFertAppMethod               0
FirstTopDressFert                485
MineralFertAppMethod.1           481
Harv_method                        0
Harv_hand_rent                   252
Threshing_method                   0
Residue_length                     0
Residue_perc                       0


In [14]:
# Drop rows with any null values
df_cleaned = df.dropna()

In [15]:
# Check for null values in each column
null_values = df_cleaned.isnull().sum()

# Display the result
print("Number of null values in each column:")
print(null_values)

Number of null values in each column:
ID                                    0
District                              0
Block                                 0
CultLand                              0
CropCultLand                          0
LandPreparationMethod                 0
CropTillageDate                       0
CropTillageDepth                      0
CropEstMethod                         0
SeedingSowingTransplanting            0
SeedlingsPerPit                       0
NursDetFactor                         0
TransDetFactor                        0
TransplantingIrrigationHours          0
TransplantingIrrigationSource         0
TransplantingIrrigationPowerSource    0
TransIrriCost                         0
StandingWater                         0
OrgFertilizers                        0
Ganaura                               0
CropOrgFYM                            0
PCropSolidOrgFertAppMethod            0
NoFertilizerAppln                     0
CropbasalFerts                        0
Ba

In [16]:
df_cleaned.head()

,ID,District,Block,CultLand,CropCultLand,LandPreparationMethod,CropTillageDate,CropTillageDepth,CropEstMethod,SeedingSowingTransplanting,...,2appDaysUrea,MineralFertAppMethod.1,Harv_method,Harv_hand_rent,Threshing_method,Residue_length,Residue_perc,Stubble_use,Acre,Yield
59,ID_BLJYBOIT6Z3D,Vaishali,Chehrakala,22,22,WetTillagePuddling TractorPlough FourWheelTrac...,2022-07-23,3,Manual_PuddledRandom,2022-08-22,...,30.0,Broadcasting,hand,600.0,machine,28,10,plowed_in_soil,0.227273,300
257,ID_P0BUVAKNFYB1,Vaishali,Chehrakala,15,15,WetTillagePuddling TractorPlough FourWheelTrac...,2022-06-25,2,Manual_PuddledRandom,2022-08-20,...,30.0,Broadcasting,hand,900.0,machine,26,10,plowed_in_soil,0.181818,240
467,ID_88FLSBSIIQR7,Vaishali,Chehrakala,2,2,WetTillagePuddling TractorPlough FourWheelTrac...,2022-07-18,3,Manual_PuddledRandom,2022-08-18,...,30.0,Broadcasting,hand,300.0,hand,27,10,plowed_in_soil,0.090909,110
1148,ID_5BX3A64YWEB2,Vaishali,Chehrakala,2,2,WetTillagePuddling TractorPlough FourWheelTrac...,2022-06-14,3,Manual_PuddledRandom,2022-07-16,...,30.0,Broadcasting,hand,300.0,hand,28,10,plowed_in_soil,0.090909,120
1403,ID_57V19HE06GWS,Vaishali,Chehrakala,30,10,WetTillagePuddling TractorPlough FourWheelTrac...,2022-07-19,5,Manual_PuddledRandom,2022-07-25,...,10.0,Broadcasting,hand,2.0,machine,20,10,plowed_in_soil,0.227273,400


In [17]:
# Split data for training and local testing

X = df_cleaned.drop(['ID', 'Yield'], axis = 1)
X =X.select_dtypes(include=np.number)
y = df_cleaned.Yield

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 1234)

# Instantiate model
model = RandomForestRegressor(random_state = 1234)

# Fit model
model.fit(X_train.fillna(0), y_train)

# Make predictions
preds = model.predict(X_test.fillna(0))

# Measure model performance(root mean squared error)
mean_squared_error(y_test, preds, squared=False)

c:\Users\Pranav\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


33.24132969662916

In [18]:
X_test.iloc[0]

CultLand                         15.000000
CropCultLand                     15.000000
CropTillageDepth                  2.000000
SeedlingsPerPit                   1.000000
TransplantingIrrigationHours      3.000000
TransIrriCost                   900.000000
StandingWater                     2.000000
Ganaura                           1.000000
CropOrgFYM                        1.000000
NoFertilizerAppln                 3.000000
BasalDAP                         12.000000
BasalUrea                        12.000000
1tdUrea                          12.000000
1appDaysUrea                     15.000000
2tdUrea                          12.000000
2appDaysUrea                     30.000000
Harv_hand_rent                  300.000000
Residue_length                   26.000000
Residue_perc                     10.000000
Acre                              0.272727
Name: 3332, dtype: float64

In [19]:
import joblib

# Save model
joblib.dump(model, 'yieldModel.pkl')

['yieldModel.pkl']

In [18]:
df_cleaned.columns

Index(['ID', 'District', 'Block', 'CultLand', 'CropCultLand',
       'LandPreparationMethod', 'CropTillageDate', 'CropTillageDepth',
       'CropEstMethod', 'SeedingSowingTransplanting', 'SeedlingsPerPit',
       'NursDetFactor', 'TransDetFactor', 'TransplantingIrrigationHours',
       'TransplantingIrrigationSource', 'TransplantingIrrigationPowerSource',
       'TransIrriCost', 'StandingWater', 'OrgFertilizers', 'Ganaura',
       'CropOrgFYM', 'PCropSolidOrgFertAppMethod', 'NoFertilizerAppln',
       'CropbasalFerts', 'BasalDAP', 'BasalUrea', 'MineralFertAppMethod',
       'FirstTopDressFert', '1tdUrea', '1appDaysUrea', '2tdUrea',
       '2appDaysUrea', 'MineralFertAppMethod.1', 'Harv_method',
       'Harv_hand_rent', 'Threshing_method', 'Residue_length', 'Residue_perc',
       'Stubble_use', 'Acre', 'Yield'],
      dtype='object')

In [19]:
X.columns

Index(['CultLand', 'CropCultLand', 'CropTillageDepth', 'SeedlingsPerPit',
       'TransplantingIrrigationHours', 'TransIrriCost', 'StandingWater',
       'Ganaura', 'CropOrgFYM', 'NoFertilizerAppln', 'BasalDAP', 'BasalUrea',
       '1tdUrea', '1appDaysUrea', '2tdUrea', '2appDaysUrea', 'Harv_hand_rent',
       'Residue_length', 'Residue_perc', 'Acre'],
      dtype='object')

In [20]:
df_test_cleaned.columns

Index(['ID', 'District', 'Block', 'CultLand', 'CropCultLand',
       'LandPreparationMethod', 'CropTillageDate', 'CropTillageDepth',
       'CropEstMethod', 'SeedingSowingTransplanting', 'SeedlingsPerPit',
       'NursDetFactor', 'TransDetFactor', 'TransplantingIrrigationHours',
       'TransplantingIrrigationSource', 'StandingWater', 'NoFertilizerAppln',
       'CropbasalFerts', 'MineralFertAppMethod', 'FirstTopDressFert',
       'MineralFertAppMethod.1', 'Harv_method', 'Harv_hand_rent',
       'Threshing_method', 'Residue_length', 'Residue_perc', 'Stubble_use',
       'Acre'],
      dtype='object')

In [21]:
# Make predictions on the Zindi test set
test_df = test[X.columns]
preds = model.predict(test_df.fillna(0))

# Create submisiion file to be uploaded to Zindi for scoring
sub = pd.DataFrame({'ID': df_test_cleaned.ID, 'Yield': preds})
sub.to_csv('BenchmarkSubmission.csv', index = False)

sub.head()

,ID,Yield
0,ID_F9XXEXN2ADR2,257.7
1,ID_SO3VW2X4QO93,257.9
2,ID_UKUQ7JM8E894,303.0
3,ID_QUISMWEZR2H4,294.9
4,ID_25JGI455VKCZ,328.4


In [22]:
# Create a dictionary to store user inputs
user_inputs = {}

# Iterate through the columns
columns = [
'CultLand', 'CropCultLand', 'CropTillageDepth', 'SeedlingsPerPit',
       'TransplantingIrrigationHours', 'TransIrriCost', 'StandingWater',
       'Ganaura', 'CropOrgFYM', 'NoFertilizerAppln', 'BasalDAP', 'BasalUrea',
       '1tdUrea', '1appDaysUrea', '2tdUrea', '2appDaysUrea', 'Harv_hand_rent',
       'Residue_length', 'Residue_perc', 'Acre'
]

for column in columns:
    # Take user input for each column
    user_input = input(f"Enter value for column '{column}': ")
    
    # Add user input to the dictionary
    user_inputs[column] = user_input

# Display the collected user inputs
print("\nUser Inputs:")
for key, value in user_inputs.items():
    print(f"{key}: {value}")

KeyboardInterrupt: Interrupted by user

In [ ]:
def predict_yield(user_inputs):

    import pandas as pd
    user_data = pd.DataFrame([user_inputs])

    # Use the 'predict' method of the model to make predictions
    yield_estimate = model.predict(user_data)
    return yield_estimate

# Predict yield based on user inputs
yield_estimate = predict_yield(user_inputs)

In [21]:
model.predict(X_test.iloc[0].values.reshape(1, -1))

c:\Users\Pranav\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


array([299.8])